In [ ]:
import numpy as np
N = 64;  coeff = 1.e-9;  Breg = 10000.;  Nreg = 100.

"""
Make sure N and coeffs are same as mine
Coeff needs to change U,V smaller to fit in grid
In original code, U,Vs are power of 10^{10} and this scalling pushes them into -10 to 10
"""

# reading data
list_of_strings = []
with open('data', 'r') as f:
  for line in f:
    list_of_strings.append(line)
Na = len(list_of_strings) #-1 For mine because there is extra line 
UVa, sigma = np.zeros((2*Na, 2)), np.zeros(2*Na)
Oa = np.zeros(2*Na).astype(complex)
for a in range(Na):
  current_string = (list_of_strings[-1]).split(" ") # Split on commas
  list_of_strings.pop()
  UVa[a, 0], UVa[a, 1] = float(current_string[3]), float(current_string[4])
  Oa[a] = float(current_string[5])*np.exp(1j*float(current_string[6])*np.pi/180.)
  sigma[a] = float(current_string[7])
  UVa[a + Na], Oa[a + Na], sigma[a + Na] = -UVa[a], np.conj(Oa[a]), sigma[a]
del list_of_strings, current_string

# local cubic interpolation
def cubf1(x, y):                              #      8  11
  cf1 = (1. + x - x*x)*(1. + 2.*y)*(1. - y)   #   8  2  3  10
  cf2 = (1. + y - y*y)*(1. + 2.*x)*(1. - x)   #   4  0  1  6
  return 0.5*(cf1 + cf2)*(1. - x)*(1. - y)    #      5  7
def cubf2(x, y):  return -0.5*x*(1. - x)*(1. - x)*(1. + 2.*y)*(1. - y)*(1. - y)
IND = np.array([[-2, -2], [-1, -2], [-2, -1], [-1, -1], [-3, -2], [-2, -3], \
  [0, -2], [-1, -3], [-3, -1], [-2, 0], [0, -1], [-1, 0]]).astype(int)
def cubfun12(x, y):  return np.array([cubf1(x, y), cubf1(1. - x, y), \
    cubf1(x, 1. - y), cubf1(1. - x, 1. - y), cubf2(x, y), cubf2(y, x), \
    cubf2(1. - x, y), cubf2(y, 1. - x), cubf2(x, 1. - y), cubf2(1. - y, x), \
    cubf2(1. - x, 1. - y), cubf2(1. - y, 1. - x)])

"""
Calculate DO
Compare G = -2DO
"""

# computing DO, the dirty image from observations
DOuv = np.zeros((N, N)).astype(complex)
for a in range(2*Na):
  m = np.mod(coeff*UVa[a], N);  x, y = m - np.floor(m)
  m1, m2 = np.floor(np.mod(m + np.array([2., 2.]), N)).astype(int)
  C, CUBFUN = Oa[a] / sigma[a]**2, cubfun12(x, y)
  for c in range(12):  DOuv[m1 + IND[c, 0], m2 + IND[c, 1]] += C*CUBFUN[c]
DO = N*np.real(np.fft.ifft2(DOuv))
del DOuv

# computing DK, the dirty kernel
DK49, DK = np.zeros((N, N, 7, 7)), []
for a in range(2*Na):
  m = np.mod(coeff*UVa[a], N);  x, y = m - np.floor(m)
  m1, m2 = np.floor(np.mod(m + np.array([2., 2.]), N)).astype(int)
  C, CUBFUN = 1. / sigma[a]**2, cubfun12(x, y)
  for c in range(12):
    for cp in range(12):
      DK49[m1 + IND[c, 0], m2 + IND[c, 1], 3 + IND[cp, 0] - IND[c, 0], \
           3 + IND[cp, 1] - IND[c, 1]] += CUBFUN[c]*CUBFUN[cp]*C
for u in range(N):
  for v in range(N):
    for du in range(7):
      for dv in range(7):
        if (DK49[u, v, du, dv] != 0.):
          DK.append([u, v, ((u + du) % N) - 3, ((v + dv) % N) - 3, \
            DK49[u, v, du, dv]])
del DK49, IND, CUBFUN

# computing DI, the dirtying of the image I
I12, Iuv = np.zeros((2, N, N)), np.zeros((N, N)).astype(complex)
DI, DIuv = np.zeros((N, N)), np.zeros((N, N)).astype(complex)
def calc_DI():
  global Iuv, DI, DIuv
  Iuv, DIuv = np.fft.fft2(I), np.zeros((N, N)).astype(complex)
  for q in DK:  DIuv[q[0], q[1]] += q[4]*Iuv[q[2], q[3]]
  DI = np.real(np.fft.ifft2(DIuv))
  return

# computing G, the gradient of the loss function
def neighbor(Iat, Inear):  return np.sign(Iat - Inear)
def calc_G():
  global G
  calc_DI()
  G = 2.*(DI - DO)
#  G = np.zeros((N, N))
  for i in range(N):
    for j in range(N):
      if (I[i, j] < 0.):  G[i, j] -= Breg
      if (I[i, j] > 0.):  G[i, j] += Nreg
#      G[i, j] += Breg*neighbor(I[i, j], 0.)
#      G[i, j] += Nreg*neighbor(I[i, j], I[i - 1, j])
#      G[i, j] += Nreg*neighbor(I[i, j], I[i, j - 1])
#      G[i, j] += Nreg*neighbor(I[i, j], I[(i + 1) % N, j])
#      G[i, j] += Nreg*neighbor(I[i, j], I[i, (j + 1) % N])
  return

I, dt = (0.*DO), 0.00001
for it in range(4000):
#  print(it)
  calc_G()
  I -= dt*G

#calc_DI()
for i in range(N):
  for j in range(N):
#    print(I[i, j], ' ', end='')
    print(DI[i, j] - DO[i, j], ' ', end='')
  print('')


In [5]:
import numpy as np
N = 80;  coeff = 1.e-9;  Breg = 10000.;  Nreg = 100.

"""
Make sure N and coeffs are same as mine
Coeff needs to change U,V smaller to fit in grid
In original code, U,Vs are power of 10^{10} and this scalling pushes them into -10 to 10
"""

# reading data
list_of_strings = []
with open('./data/SR1_M87_2017_095_hi_hops_netcal_StokesI.csv', 'r') as f:
  for line in f:
    list_of_strings.append(line)
list_of_strings.pop(0)
Na = len(list_of_strings) #-1 For mine because there is extra line 
print(len(list_of_strings))
UVa, sigma = np.zeros((2*Na, 2)), np.zeros(2*Na)
Oa = np.zeros(2*Na).astype(complex)
for a in range(Na):
  current_string = (list_of_strings[-1]).split(",") # Split on commas
  list_of_strings.pop()
  UVa[a, 0], UVa[a, 1] = float(current_string[3]), float(current_string[4])
  Oa[a] = float(current_string[5])*np.exp(1j*float(current_string[6])*np.pi/180.)
  sigma[a] = float(current_string[7])
  UVa[a + Na], Oa[a + Na], sigma[a + Na] = -UVa[a], np.conj(Oa[a]), sigma[a]
del list_of_strings, current_string

# local cubic interpolation
def cubf1(x, y):                              #      8  11
  cf1 = (1. + x - x*x)*(1. + 2.*y)*(1. - y)   #   8  2  3  10
  cf2 = (1. + y - y*y)*(1. + 2.*x)*(1. - x)   #   4  0  1  6
  return 0.5*(cf1 + cf2)*(1. - x)*(1. - y)    #      5  7
def cubf2(x, y):  return -0.5*x*(1. - x)*(1. - x)*(1. + 2.*y)*(1. - y)*(1. - y)
IND = np.array([[-2, -2], [-1, -2], [-2, -1], [-1, -1], [-3, -2], [-2, -3], \
  [0, -2], [-1, -3], [-3, -1], [-2, 0], [0, -1], [-1, 0]]).astype(int)
def cubfun12(x, y):  return np.array([cubf1(x, y), cubf1(1. - x, y), \
    cubf1(x, 1. - y), cubf1(1. - x, 1. - y), cubf2(x, y), cubf2(y, x), \
    cubf2(1. - x, y), cubf2(y, 1. - x), cubf2(x, 1. - y), cubf2(1. - y, x), \
    cubf2(1. - x, 1. - y), cubf2(1. - y, 1. - x)])

"""
Calculate DO
Compare G = -2DO
"""

# computing DO, the dirty image from observations
DOuv = np.zeros((N, N)).astype(complex)
for a in range(2*Na):
  m = np.mod(coeff*UVa[a], N);  x, y = m - np.floor(m)
  m1, m2 = np.floor(np.mod(m + np.array([2., 2.]), N)).astype(int)
  C, CUBFUN = Oa[a] / sigma[a]**2, cubfun12(x, y)
  for c in range(12):  DOuv[m1 + IND[c, 0], m2 + IND[c, 1]] += C*CUBFUN[c]
DO = N*np.real(np.fft.ifft2(DOuv))
del DOuv

# computing DK, the dirty kernel
DK49, DK = np.zeros((N, N, 7, 7)), []
for a in range(2*Na):
  m = np.mod(coeff*UVa[a], N);  x, y = m - np.floor(m)
  m1, m2 = np.floor(np.mod(m + np.array([2., 2.]), N)).astype(int)
  C, CUBFUN = 1. / sigma[a]**2, cubfun12(x, y)
  for c in range(12):
    for cp in range(12):
      DK49[m1 + IND[c, 0], m2 + IND[c, 1], 3 + IND[cp, 0] - IND[c, 0], \
           3 + IND[cp, 1] - IND[c, 1]] += CUBFUN[c]*CUBFUN[cp]*C
for u in range(N):
  for v in range(N):
    for du in range(7):
      for dv in range(7):
        if (DK49[u, v, du, dv] != 0.):
          DK.append([u, v, ((u + du) % N) - 3, ((v + dv) % N) - 3, \
            DK49[u, v, du, dv]])
del DK49, IND, CUBFUN

# computing DI, the dirtying of the image I
I12, Iuv = np.zeros((2, N, N)), np.zeros((N, N)).astype(complex)
DI, DIuv = np.zeros((N, N)), np.zeros((N, N)).astype(complex)
def calc_DI():
  global Iuv, DI, DIuv
  Iuv, DIuv = np.fft.fft2(I), np.zeros((N, N)).astype(complex)
  for q in DK:  DIuv[q[0], q[1]] += q[4]*Iuv[q[2], q[3]]
  DI = np.real(np.fft.ifft2(DIuv))
  return

# computing G, the gradient of the loss function
def neighbor(Iat, Inear):  return np.sign(Iat - Inear)
def calc_G():
  global G
  calc_DI()
  G = 2.*(DI - DO)
#  G = np.zeros((N, N))
  for i in range(N):
    for j in range(N):
      if (I[i, j] < 0.):  G[i, j] -= Breg
      if (I[i, j] > 0.):  G[i, j] += Nreg
#      G[i, j] += Breg*neighbor(I[i, j], 0.)
#      G[i, j] += Nreg*neighbor(I[i, j], I[i - 1, j])
#      G[i, j] += Nreg*neighbor(I[i, j], I[i, j - 1])
#      G[i, j] += Nreg*neighbor(I[i, j], I[(i + 1) % N, j])
#      G[i, j] += Nreg*neighbor(I[i, j], I[i, (j + 1) % N])
  return

I, dt = (0.*DO), 0.00001
for it in range(4000):
#  print(it)
  calc_G()
  I -= dt*G

#calc_DI()
for i in range(N):
  for j in range(N):
#    print(I[i, j], ' ', end='')
    print(DO[i, j], ' ', end='')
  print('')


6458


/var/folders/_b/trlmhkgj5xq968yccj4vtg1c0000gn/T/ipykernel_76522/224590749.py:82: RuntimeWarning: overflow encountered in scalar multiply
  for q in DK:  DIuv[q[0], q[1]] += q[4]*Iuv[q[2], q[3]]
/var/folders/_b/trlmhkgj5xq968yccj4vtg1c0000gn/T/ipykernel_76522/224590749.py:82: RuntimeWarning: invalid value encountered in scalar add
  for q in DK:  DIuv[q[0], q[1]] += q[4]*Iuv[q[2], q[3]]
/var/folders/_b/trlmhkgj5xq968yccj4vtg1c0000gn/T/ipykernel_76522/224590749.py:82: RuntimeWarning: overflow encountered in scalar add
  for q in DK:  DIuv[q[0], q[1]] += q[4]*Iuv[q[2], q[3]]


58323.23350943192  70733.3062757431  82908.68409049079  92093.29597462673  96191.64946119869  94185.45692834012  86356.83633558101  74270.97275264477  60512.68334148386  48215.84824062855  40464.49903574292  39672.091292715326  47055.28789423483  62307.38086697478  83544.85371207645  107553.02396196139  130301.15479501692  147644.07089546905  156086.74887208233  153469.44556779892  139438.83502750646  115605.40025413844  85343.50799051451  53257.965932739025  24406.30998497609  3416.220434902047  -6338.457648608193  -3347.350771767801  11758.08722193614  36311.86557677372  66086.86495389584  96044.63233192734  121219.67557209727  137572.8346386782  142657.8648843111  135985.44624820753  119030.44196667266  94898.04456427575  67727.82672039623  41958.97089525352  21597.07612844174  9611.179758738894  7553.565962123295  15444.08987812035  31907.136575241424  54504.54273106925  80180.36084271292  105726.973942147  128195.25483502369  145198.29200581938  155090.23491747264  157030.43856733

In [6]:
DO*-2

array([[-116646.46701886, -141466.61255149, -165817.36818098, ...,
         -91946.83284852,  -87843.82646929,  -97128.13828764],
       [-165045.82106368, -178843.11623055, -186713.53413196, ...,
        -136597.50612495, -139412.16678247, -150238.93347765],
       [-223375.25841561, -224660.28322895, -214387.37467335, ...,
        -190591.69404424, -201703.64552191, -214172.77495052],
       ...,
       [ -54179.07539426,  -78038.92143613, -114863.56670442, ...,
         -83133.0132055 ,  -57237.93638735,  -46941.85795766],
       [ -62224.78847647,  -93770.16616968, -134309.62183714, ...,
         -62214.2329105 ,  -44861.99881259,  -44933.17333172],
       [ -82189.65065602, -113677.88005881, -149700.23196057, ...,
         -65393.96363538,  -54949.31603845,  -61180.55395091]])

In [7]:
G1 = np.loadtxt("./gradient_2.txt", delimiter=",")
G2 = np.loadtxt("./gradient_forward.txt", delimiter=",")

In [8]:
np.max(np.abs(G1+2*DO))

2116612.9597594156

In [9]:
np.max(np.abs(G2+2*DO))

7668863.636313394